# クラスの構造体的な使い方

よく、Fortran出身の人のプログラムでは以下のような記述が見られます。

In [1]:
n = 4
x = [ 1, 2, 3, 4 ]
y = [ 5, 6, 7, 8 ]

In [2]:
z = [ 0, 0, 0, 0 ]
for i in range(n):
    z[i] = x[i] + y[i]

In [3]:
print(z)

[6, 8, 10, 12]


特に前進差分などの計算では頻発します。例えば、  
lambda[i]  
crow[i]  ...etc...
のように。これは、本当に見づらいです。また、indexのずれなど、予期せぬバグを書いてしまうことになります。  
さらに、i = 0の場合は空気だよ・・・みたいな暗黙の決めごとが多発して、他人にとって非常に読みづらい、つまりプログラムの再利用性が著しく低下し、「結局は書いた人に聞かないと何をやっているかわからない。」というプログラムになってしまいます。
良いプログラムは説明不要で、見ただけで、すっきりと流れるように頭の中に入ってくるプログラムです。
次のように書くことで、非常にプログラムがすっきりします。

In [4]:
class Cell():
    def __init__( self, x, y):
        self.x = x
        self.y = y

構造体的な使い方で、例えば、以下のように使えます。

In [5]:
c1 = Cell( 2, 3 )

In [6]:
c1.x

2

In [7]:
c1.y

3

書き換えも自由

In [8]:
c1.x = 5

In [9]:
c1.x

5

内部パラメータが最初に決まらなかったら・・・

In [10]:
c2 = Cell( None, None )
c2.x = 1
c2.y = 2

In [11]:
c2.x

1

In [12]:
c2.y

2

とします。  
（ただし、最初に決まらないということはそもそも変数で表すのではなくて、式で表すことができるので、基本的には最初にNoneを使わないといけないということは構造的になにか問題を抱えているということになるんですけどね・・・。）

構造体を使う利点は、xとかyとかが、それだけでは意味をなさず、合わさってはじめて意味を成すということが一目瞭然になることです。

これを応用して最初の例を書き換えると以下のようになります。

In [13]:
# x = [ 1, 2, 3, 4 ]
# y = [ 5, 6, 7, 8 ]
cells = [ Cell(1,5), Cell(2,6), Cell(3,7), Cell(4,8) ]

最初の代入は面倒くさいのですが、一度、値を格納してしまうと、次からのコードが驚くほど簡単かつ直感的に書けます。

In [14]:
for cell in cells:
    cell.z = cell.x + cell.y

In [15]:
cells[0].z, cells[1].z, cells[2].z, cells[3].z

(6, 8, 10, 12)

ここまでの知識でも随分とコードがすっきりします。C言語系では、これを構造体的な使い方といいます。  
まずは、ここまでを目指してください。多分このレベルだったら学部生でも到達できるはず。  
次に、この構造体的な考えに慣れてきたら、パラメータの塊である構造体にメソッドを追加します。これをクラスといいます。

クラス　＝　構造体　＋　メソッド

クラスにメソッドを追加し、シンプルにします。

In [16]:
class Cell2():
    def __init__( self, x, y ):
        self.x = x
        self.y = y
    def AddXandY( self ):
        self.z = self.x + self.y

In [17]:
cells2 = [ Cell2(1,5), Cell2(2,6), Cell2(3,7), Cell2(4,8) ]

中身はこんなかんじ（まだ、演算していません。）

In [18]:
for c in cells2:
    print(vars(c))

{'x': 1, 'y': 5}
{'x': 2, 'y': 6}
{'x': 3, 'y': 7}
{'x': 4, 'y': 8}


こうしてやると、変数すら書かなくてよくなります。こういうふうに・・・

In [19]:
for cell2 in cells2:
    cell2.AddXandY()

中身はこうなりました。パラメータ'z'が追加されています。

In [20]:
for c in cells2:
    print(vars(c))

{'x': 1, 'y': 5, 'z': 6}
{'x': 2, 'y': 6, 'z': 8}
{'x': 3, 'y': 7, 'z': 10}
{'x': 4, 'y': 8, 'z': 12}


z が追加されましたね。

もう少し、熱移動に即して、具体例を示します。

実際には以下のように書きます。

１つのセルがあったとします。

In [21]:
class Cell3():
    def __init__( self, initialtemp, cd_mns, cd_pls, cap ):
        self.temp = initialtemp # 温度の初期値(℃)
        self.cd_mns = cd_mns # セルがもつコンダクタンス(-x側)(W/m2K)
        self.cd_pls = cd_pls # セルがもつコンダクタンス(-x側)(W/m2K)
        self.cap  = cap # セルのキャパシティー(J/m3K)
    def makeCombinedConductance( self, cd_mnsside, cd_plsside ):
        self.cc_mns = 1 / ( 1 / cd_mnsside + 1 / self.cd_mns ) # 隣のセルのコンダクタンスとの合成
        self.cc_pls = 1 / ( 1 / cd_plsside + 1 / self.cd_pls ) # 隣のセルのコンダクタンスとの合成
        # 1度抵抗にしてから合計し再度コンダクタンスにしている
    def newtempcalc( self, temp_mnsside, temp_plsside, deltaT ):
        # 流入する熱流の計算(W/m2)
        heatflow = ( ( temp_mnsside - self.temp ) * self.cc_mns + ( temp_plsside - self.temp ) * self.cc_pls ) * deltaT
        # 新しい温度(℃)
        self.newtemp = heatflow / self.cap + self.temp
    def updatetemp( self ):
        # 古い温度を新しい温度に置き換え
        self.temp = self.newtemp

In [22]:
c3 = Cell3( initialtemp = 0.0, cd_mns = 1.0, cd_pls = 1.0, cap = 100.0 )

In [23]:
c3.makeCombinedConductance( cd_mnsside = 1.0, cd_plsside = 1.0 )

In [24]:
c3.newtempcalc( temp_mnsside = 5.0, temp_plsside = 5.0, deltaT = 1.0 )

In [25]:
c3.updatetemp()

In [26]:
c3.temp

0.05

セルのコンダクタンスが中心点から-x側、+x側ともに1.0(W/m2K)  
となりのセルも両側とも1.0(W/mK)  
したがって合成コンダクタンス$CC$は両側ともに、  
$ CC = \frac{ 1 }{ \frac{ 1 }{ 1.0 } + \frac{ 1 }{ 1.0 } } = 0.5 $ (W/m2K)  
両側の温度はともに5℃だから、温度差は5(K)。したがって、熱流fは、  
$ f_{mns} = 0.5 \times 5.0 \times 1.0 = 2.5 $ (J/m2)  
$ f_{pls} = 0.5 \times 5.0 \times 1.0 = 2.5 $ (J/m2)  
あわせて、  
$ f = f_{mns} + f_{pls} = 2.5 + 2.5 = 5.0 $ (J/m2)  
温度上昇$\Delta T $は、  
$ \Delta T = \frac{ 5.0 }{ 100.0 } = 0.05 $ (℃)  
となり、計算はあっていることになります。

実際には、セルはたくさんあるので、

初期化。ここでは、セルの数を10個にしました。

In [27]:
cells = [ Cell3( initialtemp = 0.0, cd_mns = 1.0, cd_pls = 1.0, cap = 100.0 ) for i in range(10) ]

合成コンダクタンスの作成

In [28]:
N = len(cells)
for i in range(N):
    if i == 0: # セルが端っこの場合は隣のセルが定義されていないので・・・
        cellmns_cd_pls = 1.0  # なにか表面熱伝達率のようなものを別途与えてあげる
    else:
        cellmns_cd_pls = cells[i-1].cd_pls
    if i == N-1: # こちらも同じ
        cellpls_cd_mns = 1.0
    else:
        cellpls_cd_mns = cells[i+1].cd_mns
    cells[i].makeCombinedConductance( cd_mnsside = cellmns_cd_pls, cd_plsside = cellpls_cd_mns )

中身はこんな感じ

x-側のコンダクタンス(cc_mns)

In [29]:
[ c.cc_mns for c in cells ]

[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

x+側のコンダクタンス(cc_pls)

In [30]:
[ c.cc_pls for c in cells ]

[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

In [31]:
def cells_get_newtemp( deltaT ):
    for i in range(N):
        if i == 0: # セルが端っこの場合は隣のセルが定義されていないので・・・
            temp_mns = 5.0  # なにか室温or外気温のようなものを別途与えてあげる
        else:
            temp_mns = cells[i-1].temp
        if i == N-1: # こちらも同じ
            temp_pls = 5.0
        else:
            temp_pls = cells[i+1].temp
        cells[i].newtempcalc( temp_mnsside = temp_mns, temp_plsside = temp_pls, deltaT = deltaT )
    for i in range(N):
        cells[i].updatetemp()

これをぐるぐるまわします。

In [32]:
for i in range(3600):
    cells_get_newtemp( deltaT = 1.0 )

結果は

In [33]:
for c in cells:
    print ( c.temp )

4.585697347725006
4.20496263589897
3.88864184264323
3.662359255642234
3.54444302315501
3.54444302315501
3.662359255642234
3.88864184264323
4.20496263589897
4.585697347725006


3600秒まわした結果、両側が5℃に対して、大分温度が上昇してきました。両側は4.5℃まで温まりましたが、中心部は3.5℃ぐらいまでしかあがっていないことがわかります。

ただし、クラスの概念が研究室に浸透するまでは、とりあえずは構造体の概念にとどめておくのも策かと思います。壁だけの計算であれば、構造体だけでも質の高いプログラムは記述可能です。ただし、壁と室というように複合させて計算しようとすると、クラスの概念がないと、かなり厳しいかと思います。